# Setup Variables

In [1]:
aws_profile_name = "dev"
aws_region = "us-east-1"

In [2]:
import boto3

boto3.setup_default_session(profile_name=aws_profile_name)
BEDROCK_CLIENT = boto3.client("bedrock-runtime", aws_region)

In [3]:
from langchain_aws import ChatBedrockConverse

model = ChatBedrockConverse(
    client=BEDROCK_CLIENT,
    model="anthropic.claude-3-5-sonnet-20240620-v1:0",
    max_tokens=1000, 
    temperature=0.2,
    top_p=0.9,
)

In [4]:
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

@tool(parse_docstring=True)
def search(query: str) -> str:
    """Call to surf the web.
    
    Args:
        query: The query you would like to use when searching the web.
    """
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."


tools = [search]

tool_node = ToolNode(tools)


In [5]:
import json

schemas = [t.get_input_jsonschema() for t in tools]
tools_string = [
f"""
Tool Name: {schema["title"]}
Tool Description: {schema["description"]}
Required Parameters: {schema["required"]}
Parameters: {json.dumps(schema["properties"], indent=2)}
"""
for schema in schemas
]

print("\n\n".join(tools_string))


Tool Name: search
Tool Description: Call to surf the web.
Required Parameters: ['query']
Parameters: {
  "query": {
    "description": "The query you would like to use when searching the web.",
    "title": "Query",
    "type": "string"
  }
}



In [6]:
from langgraph.graph import END, START, StateGraph, MessagesState
from typing import Annotated, Literal, TypedDict, cast
from langchain_core.messages import AIMessage

def call_model(state: MessagesState):
    messages = state['messages']
    model_with_tools = model.bind_tools(tools)
    response = model_with_tools.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


def should_call_tool_or_end(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = cast(AIMessage, messages[-1])
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        print(f"Calling tool: {last_message.tool_calls}")
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END



In [7]:
# Define a new graph
graph = StateGraph(MessagesState)

# Define the two nodes we will cycle between
graph.add_node("agent", call_model)
graph.add_node("tools", tool_node)

graph.set_entry_point("agent")
graph.add_conditional_edges(
    "agent",
    should_call_tool_or_end,
)
graph.add_edge("tools", 'agent')

agent = graph.compile()

In [8]:
agent.get_graph().print_ascii()

        +-----------+         
        | __start__ |         
        +-----------+         
              *               
              *               
              *               
          +-------+           
          | agent |           
          +-------+           
         *         .          
       **           ..        
      *               .       
+-------+         +---------+ 
| tools |         | __end__ | 
+-------+         +---------+ 


In [9]:
from langchain.globals import set_verbose, set_debug

set_debug(False)
set_verbose(False)

from langchain_core.messages import HumanMessage, SystemMessage

output = agent.invoke({
    "messages": [
        SystemMessage("""
            Use tools when needed instead of asking permission.
            Don't reference the tool use in your response content.
        """),
        HumanMessage("what is the weather in Urbandale, IA?")
    ]
})

Calling tool: [{'name': 'search', 'args': {'query': 'current weather in Urbandale, IA'}, 'id': 'tooluse_xi44KbI5QBOQNu3Jo7OA6w', 'type': 'tool_call'}]


In [10]:
for message in output["messages"]:
    print(message)

content="\n            Use tools when needed instead of asking permission.\n            Don't reference the tool use in your response content.\n        " additional_kwargs={} response_metadata={} id='834005ff-fcdf-4812-8eda-b745b4bbe86e'
content='what is the weather in Urbandale, IA?' additional_kwargs={} response_metadata={} id='47b5dff3-acc9-483b-ac44-c11f89ea7dc3'
content=[{'type': 'text', 'text': "To get the current weather information for Urbandale, IA, I'll need to search for the most up-to-date weather data. Let me do that for you."}, {'type': 'tool_use', 'name': 'search', 'input': {'query': 'current weather in Urbandale, IA'}, 'id': 'tooluse_xi44KbI5QBOQNu3Jo7OA6w'}] additional_kwargs={} response_metadata={'ResponseMetadata': {'RequestId': '34d771e0-8537-452a-8682-a9991e4b65b1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 17 Oct 2024 19:44:23 GMT', 'content-type': 'application/json', 'content-length': '451', 'connection': 'keep-alive', 'x-amzn-requestid': '34d771e0-853

In [11]:
print(output["messages"][-1].content)

Based on the search results, I can provide you with the current weather information for Urbandale, IA:

The current temperature in Urbandale, IA is 90 degrees Fahrenheit (32.2°C), and it's sunny.

This indicates that Urbandale is experiencing warm summer weather today. It's quite hot outside, so if you're in the area or planning to visit, it would be a good idea to stay hydrated, use sun protection, and try to stay cool. The sunny conditions make it a good day for outdoor activities, but remember to take precautions against the heat, especially during the hottest parts of the day.

Is there anything else you'd like to know about the weather in Urbandale or any other location?
